# SLAMseq

## Installing slamdunk (only do once)

In [1]:
# install slamdunk
#! pip3 install git+https://github.com/jkobject/slamdunk.git --upgrade

In [2]:
# clone and install genepy
# pip install -e genepy

In [3]:
# please also install cutadapt, fastqc, and trimgalore
#! conda install -c bioconda cutadapt
#! conda install -c bioconda fastqc
#! conda install -c bioconda trim-galore

In [1]:
from __future__ import print_function
import os.path
import pandas as pd
import sys
sys.path.insert(0, '../../')
import seaborn as sns
import numpy as np
from natsort import os_sorted

from genepy.utils import helper as h
from genepy.utils import plot as genepyPlot
from genepy.rna import pyDESeq2
import genepy.rna as rna

In [3]:
from bokeh.plotting import *
from bokeh.models import HoverTool
from bokeh.io import output_notebook

import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.manifold import MDS, TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
#from umap import UMAP

output_notebook()
%load_ext autoreload
%matplotlib inline
%autoreload 2
%load_ext rpy2.ipython

Loading BokehJS ...

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [4]:
def createDir(directory):
    if not os.path.exists(directory):
        print("Creating output directory: " + directory)
        os.makedirs(directory)

# MV411 WT and MYB-dTAG cells co-cultured

SLAMseq with WT and MYB-dTAG MV411 cells co-cultured
<br>12h with drug + 1h 4sU with 0.5$\mu$M VHL (with ERCC spike-in control)
<br>__Processed without alt loci__

In [5]:
project = "MYB_degraded_coculture_v1"

## download the data

In [6]:
location = "../data/slamseq_"+project+"/" # new data location
loc = "220526_MP10028_fastq" # transfer AML folder

In [17]:
! mkdir ../$location
! mkdir ../results/slamseq_$project
! mkdir $location && mkdir $location/qc # make data dir in AMLproject
! mkdir ../$location/fastqs/ # make dir in /home/monika/data/project/fastqs

mkdir: cannot create directory ‘../../data/slamseq_MYB_degron_coculture_v1/’: File exists
mkdir: cannot create directory ‘../results/slamseq_MYB_degron_coculture_v1’: File exists
mkdir: cannot create directory ‘../data/slamseq_MYB_degron_coculture_v1/’: File exists
mkdir: cannot create directory ‘../../data/slamseq_MYB_degron_coculture_v1//fastqs/’: File exists


In [19]:
! gsutil ls gs://transfer-amlproject/$loc/*.fastq.gz

gs://transfer-amlproject/220526_MP10028_fastq/20220526_10_MP10028_S34_R1_001.fastq.gz
gs://transfer-amlproject/220526_MP10028_fastq/20220526_10_MP10028_S34_R2_001.fastq.gz
gs://transfer-amlproject/220526_MP10028_fastq/20220526_11_MP10028_S35_R1_001.fastq.gz
gs://transfer-amlproject/220526_MP10028_fastq/20220526_11_MP10028_S35_R2_001.fastq.gz
gs://transfer-amlproject/220526_MP10028_fastq/20220526_12_MP10028_S36_R1_001.fastq.gz
gs://transfer-amlproject/220526_MP10028_fastq/20220526_12_MP10028_S36_R2_001.fastq.gz
gs://transfer-amlproject/220526_MP10028_fastq/20220526_13_MP10028_S37_R1_001.fastq.gz
gs://transfer-amlproject/220526_MP10028_fastq/20220526_13_MP10028_S37_R2_001.fastq.gz
gs://transfer-amlproject/220526_MP10028_fastq/20220526_15_MP10028_S38_R1_001.fastq.gz
gs://transfer-amlproject/220526_MP10028_fastq/20220526_15_MP10028_S38_R2_001.fastq.gz
gs://transfer-amlproject/220526_MP10028_fastq/20220526_16_MP10028_S39_R1_001.fastq.gz
gs://transfer-amlproject/220526_MP10028_fastq/20220526

In [21]:
# transfer qc files
! gsutil -m cp gs://transfer-amlproject/$loc/multiqc_report.html $location/qc/
! gsutil -m cp -r gs://transfer-amlproject/$loc/Reports/ $location/qc/
! gsutil -m cp -r gs://transfer-amlproject/$loc/multiqc_data/ $location/qc/

Copying gs://transfer-amlproject/220526_MP10028_fastq/multiqc_report.html...
/ [1/1 files][  1.5 MiB/  1.5 MiB] 100% Done                                    
Operation completed over 1 objects/1.5 MiB.                                      
Copying gs://transfer-amlproject/220526_MP10028_fastq/Reports/20220526_10_MP10028_S34_R1_001_fastqc.html...
Copying gs://transfer-amlproject/220526_MP10028_fastq/Reports/20220526_10_MP10028_S34_R1_001_fastqc.zip...
Copying gs://transfer-amlproject/220526_MP10028_fastq/Reports/20220526_10_MP10028_S34_R2_001_fastqc.html...
Copying gs://transfer-amlproject/220526_MP10028_fastq/Reports/20220526_10_MP10028_S34_R2_001_fastqc.zip...
Copying gs://transfer-amlproject/220526_MP10028_fastq/Reports/20220526_11_MP10028_S35_R1_001_fastqc.html...
Copying gs://transfer-amlproject/220526_MP10028_fastq/Reports/20220526_11_MP10028_S35_R1_001_fastqc.zip...
Copying gs://transfer-amlproject/220526_MP10028_fastq/Reports/20220526_11_MP10028_S35_R2_001_fastqc.zip...
Copying 

- [76/76 files][ 35.3 MiB/ 35.3 MiB] 100% Done                                  
Operation completed over 76 objects/35.3 MiB.                                    
Copying gs://transfer-amlproject/220526_MP10028_fastq/multiqc_data/multiqc.log...
Copying gs://transfer-amlproject/220526_MP10028_fastq/multiqc_data/multiqc_data.json...
Copying gs://transfer-amlproject/220526_MP10028_fastq/multiqc_data/multiqc_fastqc.txt...
Copying gs://transfer-amlproject/220526_MP10028_fastq/multiqc_data/multiqc_general_stats.txt...
Copying gs://transfer-amlproject/220526_MP10028_fastq/multiqc_data/multiqc_qualimap_bamqc_genome_results.txt...
Copying gs://transfer-amlproject/220526_MP10028_fastq/multiqc_data/multiqc_sources.txt...
/ [6/6 files][  2.3 MiB/  2.3 MiB] 100% Done                                    
Operation completed over 6 objects/2.3 MiB.                                      


In [22]:
! cp $location/qc/multiqc_report.html ../results/slamseq_$project

In [23]:
fastqs = ! gsutil ls gs://transfer-amlproject/$loc/*.gz

add sample names to AML sample tracker google sheet (RNA_tracker_JK)

missing file: __20220526_14_MP10028__ failed initial QC

In [26]:
fastqs = [fastq.split("/")[-1] for fastq in os_sorted(fastqs)]
print(len(fastqs))
# add sample names to AML sample tracker google sheet
for val1, val2 in h.grouped(fastqs, 2):
    print(val1.split("_R")[0])

38
20220526_1_MP10028_S25
20220526_2_MP10028_S26
20220526_3_MP10028_S27
20220526_4_MP10028_S28
20220526_5_MP10028_S29
20220526_6_MP10028_S30
20220526_7_MP10028_S31
20220526_8_MP10028_S32
20220526_9_MP10028_S33
20220526_10_MP10028_S34
20220526_11_MP10028_S35
20220526_12_MP10028_S36
20220526_13_MP10028_S37
20220526_15_MP10028_S38
20220526_16_MP10028_S39
20220526_17_MP10028_S40
20220526_18_MP10028_S41
20220526_19_MP10028_S42
20220526_20_MP10028_S43


In [30]:
# transfer fastqs
! gsutil -m cp gs://transfer-amlproject/$loc/*.fastq.gz ../$location/fastqs/

Copying gs://transfer-amlproject/220526_MP10028_fastq/20220526_11_MP10028_S35_R1_001.fastq.gz...
Copying gs://transfer-amlproject/220526_MP10028_fastq/20220526_8_MP10028_S32_R1_001.fastq.gz...
Copying gs://transfer-amlproject/220526_MP10028_fastq/20220526_2_MP10028_S26_R2_001.fastq.gz...
Copying gs://transfer-amlproject/220526_MP10028_fastq/20220526_17_MP10028_S40_R2_001.fastq.gz...
Copying gs://transfer-amlproject/220526_MP10028_fastq/20220526_1_MP10028_S25_R1_001.fastq.gz...
Copying gs://transfer-amlproject/220526_MP10028_fastq/20220526_6_MP10028_S30_R2_001.fastq.gz...
Copying gs://transfer-amlproject/220526_MP10028_fastq/20220526_11_MP10028_S35_R2_001.fastq.gz...
Copying gs://transfer-amlproject/220526_MP10028_fastq/20220526_5_MP10028_S29_R1_001.fastq.gz...
Copying gs://transfer-amlproject/220526_MP10028_fastq/20220526_17_MP10028_S40_R1_001.fastq.gz...
Copying gs://transfer-amlproject/220526_MP10028_fastq/20220526_15_MP10028_S38_R1_001.fastq.gz...
Copying gs://transfer-amlproject/22

## rename local fastqs

In [45]:
fastq_folder = "../"+location+"fastqs"
print(fastq_folder)
fastqs = ! ls $fastq_folder
fastqs = os_sorted(fastqs)
fastqs

../../data/slamseq_MYB_degraded_coculture_v1/fastqs


['mr571-MV411-DMSO_coculture_12h-r1_R1_001.fastq.gz',
 'mr571-MV411-DMSO_coculture_12h-r1_R2_001.fastq.gz',
 'mr572-MV411-DMSO_coculture_12h-r2_R1_001.fastq.gz',
 'mr572-MV411-DMSO_coculture_12h-r2_R2_001.fastq.gz',
 'mr573-MV411-DMSO_coculture_12h-r3_R1_001.fastq.gz',
 'mr573-MV411-DMSO_coculture_12h-r3_R2_001.fastq.gz',
 'mr574-MV411-DMSO_coculture_12h-r4_R1_001.fastq.gz',
 'mr574-MV411-DMSO_coculture_12h-r4_R2_001.fastq.gz',
 'mr575-MV411-DMSO_coculture_12h-r5_R1_001.fastq.gz',
 'mr575-MV411-DMSO_coculture_12h-r5_R2_001.fastq.gz',
 'mr576-MV411-VHL_coculture_12h-r1_R1_001.fastq.gz',
 'mr576-MV411-VHL_coculture_12h-r1_R2_001.fastq.gz',
 'mr577-MV411-VHL_coculture_12h-r2_R1_001.fastq.gz',
 'mr577-MV411-VHL_coculture_12h-r2_R2_001.fastq.gz',
 'mr578-MV411-VHL_coculture_12h-r3_R1_001.fastq.gz',
 'mr578-MV411-VHL_coculture_12h-r3_R2_001.fastq.gz',
 'mr579-MV411-VHL_coculture_12h-r4_R1_001.fastq.gz',
 'mr579-MV411-VHL_coculture_12h-r4_R2_001.fastq.gz',
 'mr580-MV411-VHL_coculture_12h-r5_R

In [34]:
fastqs[::2]

['20220526_1_MP10028_S25_R1_001.fastq.gz',
 '20220526_2_MP10028_S26_R1_001.fastq.gz',
 '20220526_3_MP10028_S27_R1_001.fastq.gz',
 '20220526_4_MP10028_S28_R1_001.fastq.gz',
 '20220526_5_MP10028_S29_R1_001.fastq.gz',
 '20220526_6_MP10028_S30_R1_001.fastq.gz',
 '20220526_7_MP10028_S31_R1_001.fastq.gz',
 '20220526_8_MP10028_S32_R1_001.fastq.gz',
 '20220526_9_MP10028_S33_R1_001.fastq.gz',
 '20220526_10_MP10028_S34_R1_001.fastq.gz',
 '20220526_11_MP10028_S35_R1_001.fastq.gz',
 '20220526_12_MP10028_S36_R1_001.fastq.gz',
 '20220526_13_MP10028_S37_R1_001.fastq.gz',
 '20220526_15_MP10028_S38_R1_001.fastq.gz',
 '20220526_16_MP10028_S39_R1_001.fastq.gz',
 '20220526_17_MP10028_S40_R1_001.fastq.gz',
 '20220526_18_MP10028_S41_R1_001.fastq.gz',
 '20220526_19_MP10028_S42_R1_001.fastq.gz',
 '20220526_20_MP10028_S43_R1_001.fastq.gz']

In [35]:
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret_web.json', '~/.storage.json')
url = "https://docs.google.com/spreadsheets/d/1yFLjYB1McU530JnLgL0QIMAKIkVl3kl0_LCHje2gk8U/edit?ts=5fab1071#gid=738732237"
gsheet = sheets.get(url).sheets[12].to_frame()

In [38]:
# get new sample names from AML sample tracker - RNA_tracker_JK
idx = gsheet.index[gsheet['batch'] == project] # get first row with project number
sample_info = gsheet.loc[idx[0]:, ["prev_name", "name"]]
sample_info = sample_info.iloc[:int(len(fastqs)/2),]
print(sample_info.shape)

(19, 2)


In [40]:
# make dict with names
rename = dict(zip(sample_info.prev_name, sample_info.name))

# rename fastqs locally
for val in fastqs:
    rep  = val
    for old, new in rename.items():
        rep = rep.replace(old, new)
    if rep != val:
        ! mv $fastq_folder/$val $fastq_folder/$rep

# Processing

In [43]:
project

'MYB_degraded_coculture_v1'

In [46]:
fastqs = ! ls $fastq_folder/*fastq.gz
fastqs[::2]

['../../data/slamseq_MYB_degraded_coculture_v1/fastqs/mr571-MV411-DMSO_coculture_12h-r1_R1_001.fastq.gz',
 '../../data/slamseq_MYB_degraded_coculture_v1/fastqs/mr572-MV411-DMSO_coculture_12h-r2_R1_001.fastq.gz',
 '../../data/slamseq_MYB_degraded_coculture_v1/fastqs/mr573-MV411-DMSO_coculture_12h-r3_R1_001.fastq.gz',
 '../../data/slamseq_MYB_degraded_coculture_v1/fastqs/mr574-MV411-DMSO_coculture_12h-r4_R1_001.fastq.gz',
 '../../data/slamseq_MYB_degraded_coculture_v1/fastqs/mr575-MV411-DMSO_coculture_12h-r5_R1_001.fastq.gz',
 '../../data/slamseq_MYB_degraded_coculture_v1/fastqs/mr576-MV411-VHL_coculture_12h-r1_R1_001.fastq.gz',
 '../../data/slamseq_MYB_degraded_coculture_v1/fastqs/mr577-MV411-VHL_coculture_12h-r2_R1_001.fastq.gz',
 '../../data/slamseq_MYB_degraded_coculture_v1/fastqs/mr578-MV411-VHL_coculture_12h-r3_R1_001.fastq.gz',
 '../../data/slamseq_MYB_degraded_coculture_v1/fastqs/mr579-MV411-VHL_coculture_12h-r4_R1_001.fastq.gz',
 '../../data/slamseq_MYB_degraded_coculture_v1/fas

## trim_galore

In [49]:
for val1, val2 in h.grouped(fastqs, 2):
    print(os.path.basename(val1))

mr571-MV411-DMSO_coculture_12h-r1_R1_001.fastq.gz
mr572-MV411-DMSO_coculture_12h-r2_R1_001.fastq.gz
mr573-MV411-DMSO_coculture_12h-r3_R1_001.fastq.gz
mr574-MV411-DMSO_coculture_12h-r4_R1_001.fastq.gz
mr575-MV411-DMSO_coculture_12h-r5_R1_001.fastq.gz
mr576-MV411-VHL_coculture_12h-r1_R1_001.fastq.gz
mr577-MV411-VHL_coculture_12h-r2_R1_001.fastq.gz
mr578-MV411-VHL_coculture_12h-r3_R1_001.fastq.gz
mr579-MV411-VHL_coculture_12h-r4_R1_001.fastq.gz
mr580-MV411-VHL_coculture_12h-r5_R1_001.fastq.gz
mr581-MV411-MYB-dTAG-DMSO_coculture_12h-r1_R1_001.fastq.gz
mr582-MV411-MYB-dTAG-DMSO_coculture_12h-r2_R1_001.fastq.gz
mr583-MV411-MYB-dTAG-DMSO_coculture_12h-r3_R1_001.fastq.gz
mr584-MV411-MYB-dTAG-DMSO_coculture_12h-r4_R1_001.fastq.gz
mr585-MV411-MYB-dTAG-VHL_coculture_12h-r1_R1_001.fastq.gz
mr586-MV411-MYB-dTAG-VHL_coculture_12h-r2_R1_001.fastq.gz
mr587-MV411-MYB-dTAG-VHL_coculture_12h-r3_R1_001.fastq.gz
mr588-MV411-MYB-dTAG-VHL_coculture_12h-r4_R1_001.fastq.gz
mr589-MV411-MYB-dTAG-VHL_coculture_12

In [50]:
%%time
h.parrun(['trim_galore --paired --cores 8 --retain_unpaired -stringency 3\
 --illumina '+val1+' '+val2+' -o '+fastq_folder for val1, val2 in h.grouped(fastqs, 2)], 2)

CPU times: user 816 ms, sys: 364 ms, total: 1.18 s
Wall time: 9h 21min 16s


['pigz 2.6\npigz 2.6\n',
 'pigz 2.6\npigz 2.6\n',
 'pigz 2.6\npigz 2.6\n',
 'pigz 2.6\npigz 2.6\n',
 'pigz 2.6\npigz 2.6\n',
 'pigz 2.6\npigz 2.6\n',
 'pigz 2.6\npigz 2.6\n',
 'pigz 2.6\npigz 2.6\n',
 'pigz 2.6\npigz 2.6\n',
 'pigz 2.6\n']

## slamdunk

install slamdunk & dependencies if needed (see above)

In [51]:
# install slamdunk
#! pip3 install git+https://github.com/jkobject/slamdunk.git --upgrade

In [52]:
# please also install cutadapt, fastqc, and trimgalore
#! conda install -c bioconda cutadapt
#! conda install -c bioconda fastqc
#! conda install -c bioconda trim-galore

download reference genome files

In [53]:
# google storage bucket with reference files
#! gsutil ls gs://ccle_default_params

# copy reference genome files
#! gsutil -m cp gs://ccle_default_params/Homo_sapiens_assembly38* ../../ref/

# copy reference genome files
#! cp ../data/Muhar_Slamseq/{ERCC92.fa,ERCC92.gtf,GSE100708_hg38_refseq_062016_ensemblv84_3UTR.bed} ../../ref

In [7]:
# using an ERCC ref genome (you can just append ERCC fasta to the hg38 fasta)
refgenome_noAlt="../../ref/Homo_sapiens_assembly38_ERCC92_noAlt.fasta"

In [8]:
# gene intervals bed (use for all projects)
gene_intervals = "../../ref/GSE100708_hg38_refseq_062016_ensemblv84_3UTR.bed"

res = slamdunk all w/ (-fb, -m, -n 1, -rl 152, -r Homo_sapiens_assembly38_ERCC92_noAlt.fasta)

Read length note!
<br>Difference between min and max read length is > 100bp
<br>Average read length 127 (min: 8, max: 152)

In [9]:
fastq_folder = "../../data/"+"slamseq_"+project+"/fastqs"
fastqs = ! ls $fastq_folder
fastqs = [fastq for fastq in fastqs if "val" in fastq]
fastq_full = [fastq_folder+"/"+f for f in fastqs]

In [10]:
res_dir = "../../data/"+"slamseq_"+project+"/res"
res_dir

'../../data/slamseq_MYB_degraded_coculture_v1/res'

In [11]:
%%time

# print call
print(['slamdunk all'+
       ' -o '+res_dir+
       ' -r '+refgenome_noAlt+
       ' -b '+gene_intervals+
       ' -t 14'+
       ' -c 2'+
       ' -rl 152'+ # max-read-length = 152
       # new parameters
       ' -5 12'+
       ' -n 1'+ # reads map to multiple locations at once
       ' -m'+ # multimapping
       ' -N '+val1.split('/')[-1].split('_R')[0]+
       ' '+val1+" "+val2
       for val1, val2 in h.grouped(fastq_full[:2], 2)])

['slamdunk all -o ../../data/slamseq_MYB_degraded_coculture_v1/res -r ../../ref/Homo_sapiens_assembly38_ERCC92_noAlt.fasta -b ../../ref/GSE100708_hg38_refseq_062016_ensemblv84_3UTR.bed -t 14 -c 2 -rl 152 -5 12 -n 1 -m -N mr571-MV411-DMSO_coculture_12h-r1 ../../data/slamseq_MYB_degraded_coculture_v1/fastqs/mr571-MV411-DMSO_coculture_12h-r1_R1_001_val_1.fq.gz ../../data/slamseq_MYB_degraded_coculture_v1/fastqs/mr571-MV411-DMSO_coculture_12h-r1_R2_001_val_2.fq.gz']
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 196 µs


In [19]:
for val1, val2 in h.grouped(fastq_full[20:26], 2): print(os.path.basename(val1))

mr581-MV411-MYB-dTAG-DMSO_coculture_12h-r1_R1_001_val_1.fq.gz
mr582-MV411-MYB-dTAG-DMSO_coculture_12h-r2_R1_001_val_1.fq.gz
mr583-MV411-MYB-dTAG-DMSO_coculture_12h-r3_R1_001_val_1.fq.gz


In [ ]:
%%time

# run slamdunk
# we are doing it paired end
h.parrun(['slamdunk all'+
          ' -o '+res_dir+
          ' -r '+refgenome_noAlt+
          ' -b '+gene_intervals+
          ' -t 14'+
          ' -c 2'+
          ' -rl 152'+ # max-read-length = 152
          # new parameters
          ' -5 12'+
          ' -n 1'+ # reads map to multiple locations at once
          ' -m True'+ # multimapping
          ' -N '+val1.split('/')[-1].split('_R')[0]+
          ' '+val1+" "+val2+
          ' 2>&1 | tee '+res_dir+'/'+os.path.basename(val1)+'_slumdunk.log'
          for val1, val2 in h.grouped(fastq_full[20:26], 2)], 2)

## save the data to the bucket

In [ ]:
project

In [ ]:
! ls ../../data/slamseq_$project

* save fastqs to bucket
* save processed res to bucket
* remove processed fastqs from local

In [ ]:
! gsutil -m cp ../../data/slamseq_$project/fastqs/* gs://amlproject/RNA/slamseq/$project/fastqs/ 
! gsutil -m cp ../../data/slamseq_$project/res/map/* gs://amlproject/RNA/slamseq/$project/res/map/ 
! gsutil -m cp ../../data/slamseq_$project/res/filter/* gs://amlproject/RNA/slamseq/$project/res/filter/ 
! gsutil -m cp ../../data/slamseq_$project/res/snp/* gs://amlproject/RNA/slamseq/$project/res/snp/ 
! gsutil -m cp ../../data/slamseq_$project/res/count/* gs://amlproject/RNA/slamseq/$project/res/count/